In [3]:
# If not already installed, do: pip install pandas fastparquet
import pandas as pd
import fastparquet


# URLs for the datasets
URL_DATA1 = 'https://storage.data.gov.my/pricecatcher/pricecatcher_2024-05.parquet'
URL_DATA2 = 'https://storage.data.gov.my/pricecatcher/lookup_item.parquet'
URL_DATA3 = 'https://storage.data.gov.my/pricecatcher/lookup_premise.parquet'

# Load the data from each URL
df1 = pd.read_parquet(URL_DATA1)
df2 = pd.read_parquet(URL_DATA2)
df3 = pd.read_parquet(URL_DATA3)


# Convert 'date' column to datetime if it exists
if 'date' in df1.columns:
    df1['date'] = pd.to_datetime(df1['date'])


# Define the date range for filtering
start_date = '2024-05-28'
end_date = '2024-05-28'

# Filter the DataFrame based on the date range
filtered_df1 = df1[(df1['date'] >= start_date) & (df1['date'] <= end_date)]

# Assuming 'item_code' is the common column in df1 and 'item_code' in df2
# and 'item_name' is the column in df2 that you want to map
merged_df = filtered_df1.merge(df2, on='item_code', how='outer').merge(df3, on='premise_code', how='outer' )

# Selecting only specific columns
selected_columns = merged_df[['date','item_code', 'item','unit','state','premise_type', 'price']].copy()

# Strip whitespace from relevant columns
selected_columns.loc[:, 'premise_type'] = selected_columns['premise_type'].str.strip()

selected_columns.loc[:,'premise_type'] = selected_columns['premise_type'].replace({
    'Hypermarket': 'Pasar Raya',
    'Pasar Raya / Supermarket' : 'Pasar Raya'
})


# Load filter codes from the Excel file
filter_codes_df = pd.read_excel(r"C:\Users\sshilyah\Documents\Python\PPUC\KPDNHEP\Final mapping.xlsx")
filter_codes = filter_codes_df['item_code'].tolist()

# Filter the items based on the filter codes
filtered_item_codes = selected_columns[selected_columns['item_code'].isin(filter_codes)]

# Filter selected item codes and premise types
filtered_premise_types = ['Pasar Basah', 'Pasar Raya']  # Example premise types, replace with actual types

# Correct the final filtering logic
final_df = selected_columns[
    (selected_columns['item_code'].isin(filtered_item_codes['item_code'])) &  # Extract item_code
    (selected_columns['premise_type'].isin(filtered_premise_types))
]

print("Final DataFrame:")
print(final_df)

# Create a pivot table (matrix)
# For example, let's say you want to see the average price by item and state
matrix_df = final_df.pivot_table(
    index=['date','item_code','item', 'unit'],              # Rows
    columns=['state', 'premise_type'],           # Columns
    values='price',            # Values to aggregate
    aggfunc= ['mean', 'min', 'max'],            # Aggregation function
    fill_value= 0              # Fill missing values with 0
)

# Step 2: Reorder levels so that 'state' is the first, 'premise_type' second, and aggregation function last
matrix_df.columns = matrix_df.columns.reorder_levels(['state', 'premise_type', None])
# Define the desired order for the aggregation functions
state_order = ['Perlis', 'Kedah', 'Pulau Pinang', 'Perak', 'Selangor', 'W.P. Kuala Lumpur', 'W.P. Putrajaya', 'Negeri Sembilan','Melaka', 
'Johor', 'Pahang','Terengganu','Kelantan', 'Sarawak', 'Sabah', 'W.P. Labuan']
agg_order = ['mean', 'min', 'max']
# Reorder the columns at the aggregation level (level=2)
matrix_df = matrix_df.reorder_levels([0, 1, 2], axis=1)  # Ensure correct level order
matrix_df = matrix_df.sort_index(axis=1)  # Sort to ensure proper grouping
matrix_df = matrix_df.reindex(columns=state_order, level=0)
matrix_df = matrix_df.reindex(columns=agg_order, level=2)  # Reindex to order 'mean', 'min', 'max'
matrix_df = matrix_df.round(2)

# Display the updated DataFrame
#print(matrix_df)


# Save the filtered DataFrame to an Excel file
output_file = r"C:\Users\sshilyah\Documents\Python\PPUC\KPDNHEP\Output\filtered_data_280524.xlsx"
matrix_df.to_excel(output_file, index=True)

print(f"Filtered data has been saved to {output_file}")





HTTPError: HTTP Error 403: MediaTypeNotDetected